<a href="https://colab.research.google.com/github/yassine-fetoui/artificial-intelligence/blob/main/X_RAYSegmenataion_Unet_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [ ]:
!pip install -q kaggle


In [ ]:

!cp '/content/drive/MyDrive/kaggle.json' ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!mkdir ~/.kaggle
!cp '/content/drive/MyDrive/kaggle.json' ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:

!kaggle datasets download -d henriquerezermosqur/dental-x-ray-computacional-vision-segmentation

In [ ]:
!unzip /content/dental-x-ray-computacional-vision-segmentation.zip

In [ ]:
import json

# Open the file in read mode ('r')
with open('/content/train_annotations.coco.json', 'r') as f:
    # Load JSON data from the opened file
    data = json.load(f)

In [ ]:
data.keys()

In [ ]:
data['images']


In [ ]:
data.items()

In [ ]:
categories = data['categories']


In [ ]:
categories = {cat['id']: cat['name'] for cat in
data['categories']}
images = {img['id']: img['file_name'] for img in data['images']}

In [ ]:
x=images.keys


In [ ]:
x

In [ ]:
images

In [ ]:
len(images)


In [ ]:
list(categories.items())

In [ ]:
df_categories=pd.DataFrame(list(categories.items()),columns=['Category_ID',
'Category_Name'])


In [ ]:
df_categories

In [ ]:
df_images=pd.DataFrame((list(images.items())),columns=['Image_ID',
'Image_Name'])

In [ ]:
df_images.head()

In [ ]:
df_images.info()

In [ ]:
def images_mask_load(image_path,mask_path):
  image=cv2.imread(image_path)
  image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  mask=cv2.imread(mask_path,0)
  return image,mask

In [ ]:
def plot_image_mask(image,masks, categoties,category_names):
  fig, axes = plt.subplots(2, 4, figsize=(15, 15))
  for i,(image_path,mask_path,category) in enumerate(zip(images,masks,categories)):
    image,mask=images_mask_load(image_path,mask_path)
    #row=i//4
    #col=i%4
    row,colomn=divmod(i,4)
    ax_img=axes[0,colomn]
    ax_img.imshow(image)
    ax_img.set_title('Image')
    ax_img.axis('off')
    ax_img.set_title(f"Categoria: {category_names[category]}")
    ax_mask=axes[1,colomn]
    ax_mask.imshow(image)
    ax_mask.imshow(mask, cmap='jet', alpha=0.65)
    ax_mask.set_title('Mask')
    ax_mask.axis('off')
  plt.subplots_adjust(wspace=0, hspace=0)
  plt.tight_layout()
  plt.show()


In [ ]:
category_names=dict(list(categories.items())[:4])

In [ ]:
category_names

In [ ]:
categoryy=list(categories.keys())[:4]

In [ ]:
categoryy

In [ ]:
import os

In [ ]:

category_names = {0: 'vzrad2', 1: 'Caries', 2: 'Crown', 3: 'Filling'}
image_dir = '/content/Dental X_Ray/train'
mask_dir = '/content/Dental X_Ray/train/train_mask'
images=[
os.path.join(image_dir, '0a4f2d22-Hematian_Fariba_57y_31052021_132542_jpg.rf.6bfcf8cfb273a9a0767ea25499da98bc.jpg'),
os.path.join(image_dir, "3975890000-jpg_png_jpg.rf.1d1b03af3734eba73e9d12a295578027.jpg"),
os.path.join(image_dir, '4054820000-jpg_png_jpg.rf.1d4a4d0a6fd081280376d10fc085cffa.jpg'),
os.path.join(image_dir, '3794860000-jpg_png_jpg.rf.1cb8e683ddd8a1131d0ffd0c53ee4ac0.jpg')
]
masks = [
 os.path.join(mask_dir, '0a4f2d22-Hematian_Fariba_57y_31052021_132542_jpg.rf.6bfcf8cfb273a9a0767ea25499da98bc.jpg_mask.png'),
 os.path.join(mask_dir, '3975890000-jpg_png_jpg.rf.1d1b03af3734eba73e9d12a295578027.jpg_mask.png'),
 os.path.join(mask_dir, '4054820000-jpg_png_jpg.rf.1d4a4d0a6fd081280376d10fc085cffa.jpg_mask.png'),
 os.path.join(mask_dir, '3794860000-jpg_png_jpg.rf.1cb8e683ddd8a1131d0ffd0c53ee4ac0.jpg_mask.png')
]


In [ ]:
plot_image_mask(images, masks, categoryy, category_names)


In [ ]:
!pip install Pillow


In [ ]:
from PIL import Image


In [ ]:
ii=Image.open(images[0])

In [ ]:
from PIL import Image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D,Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


def load_annotaion(json_file):
  with open(json_file,"r") as f :
    annotaions=json.load(f)
  return annotaions

def load_image_and_mask(image_path, mask_path, target_size=(256,
256)):
 image = Image.open(image_path).convert('RGB')
 mask = Image.open(mask_path).convert('L')
 image = image.resize(target_size, Image.Resampling.LANCZOS)
 mask = mask.resize(target_size, Image.Resampling.LANCZOS)
 image = np.array(image)
 mask = np.array(mask)
 return image, mask




In [ ]:
image_name = 'image1.jpg'
mask_file_name = os.path.splitext(image_name)[0] + '_mask.png'
print(mask_file_name)

In [ ]:
def visiualize_batch(images, masks) :
  batch_size=len(images)
  fig,axes = plt.subplots(batch_size,2, figsize=(10,5*batch_size))

  for i in range(batch_size):
    ax_img=axes[i,0]
    ax_img.imshow(images[i])
    ax_img.axis('off')
    ax_mask=axes[i,1]
    ax_mask.imshow(masks[i], cmap='gray')
    ax_mask.axis('off')
    ax_mask.axis('off')
  plt.tight_layout()
  plt.show()

def dataset_generator(image_dir,mask_dir,annotations,batch_size,target_size=(256,256)):
  image_info=annotations['images']
  while True :
    np.random.shuffle(image_info)
    for i in range(0,len(image_info),batch_size):
      images=[]
      masks=[]
      for j in range(i,min(i+batch_size,len(image_info))):
        image_data=image_info[j]
        image_name=image_data['file_name']
        image_path=os.path.join(image_dir,image_name)
        mask_name= f"{image_name}_mask.png"
        mask_path=os.path.join(mask_dir,mask_name)
        image ,mask=load_image_and_mask(image_path,mask_path)
        images.append(image/255.0)
        masks.append(mask/255.0)
      images=np.array(images)
      masks=np.array(masks)
      yield images,masks




In [ ]:
!wget "https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png"

In [ ]:
!wget "/content/Network-architecture-of-U-Net-with-VGG16-encoder.png"

In [ ]:
from IPython.display import Image
Image(filename='/content/Network-architecture-of-U-Net-with-VGG16-encoder.png')

In [ ]:
from IPython.display import Image
Image(filename='/content/11042_2024_19022_Fig8_HTML.png')

In [ ]:
from IPython.display import Image
Image(filename='/content/11042_2024_19022_Fig8_HTML.png')

In [ ]:
def unet_VGG16_model(input_shape):


  #VGG16 Structure
  vgg16=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=input_shape))
  # include_top=False =>excluding the fully connected layers
  for layer in vgg16.layers:
    layer.trainable=False
  inputs=vgg16.input

  #x= Conv2D(64,(3,3),activation='relu',padding='same')(vgg16_output)
  #x=Conv2D(128,(3,3),activation='relu',padding='same')(x)
  cnv1=vgg16.get_layer('block1_conv2').output
  mp1=MaxPooling2D((2,2),strides=(2,2))(cnv1)
  cnv2=vgg16.get_layer('block2_conv2').output
  mp1=MaxPooling2D((2,2),strides=(2,2))(cnv2)
  cnv3=vgg16.get_layer('block3_conv3').output
  cnv4=vgg16.get_layer('block4_conv3').output
  #UNET Structre
  u5=Conv2DTranspose(256,(2,2),strides=(2,2),padding='same')(cnv4)
  u5=concatenate([u5,cnv3])
  cnv5=Conv2D(256,(3,3),activation='relu',padding='same')(u5)
  cnv5=Conv2D(256,(3,3),activation='relu',padding='same')(cnv5)
  u6=Conv2DTranspose(128,(2,2),strides=(2,2),padding='same')(cnv5)
  u6=concatenate([u6,cnv2])
  cnv6=Conv2D(128,(3,3),activation='relu',padding='same')(u6)
  cnv6=Conv2D(128,(3,3),activation='relu',padding='same')(cnv6)
  u7=Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(cnv6)
  u7=concatenate([u7,cnv1])
  cnv7=Conv2D(64,(3,3),activation='relu',padding='same')(u7)
  cnv7=Conv2D(64,(3,3),activation='relu',padding='same')(cnv7)
  outputs=Conv2D(1,(1,1),activation='sigmoid')(cnv7)
  model=Model(inputs=inputs,outputs=outputs)
  return model






In [ ]:
input_shape=(256,256,3)
model=unet_VGG16_model(input_shape)
batch_size=8

In [ ]:
train_image_dir = '/content/Dental X_Ray/train'
train_mask_dir = '/content/Dental X_Ray/train/train_mask'
train_annotation_file = '/content/train_annotations.coco.json'
valid_image_dir = '/content/Dental X_Ray/valid'
valid_mask_dir = '/content/Dental X_Ray/valid/valid_mask'
valid_annotation_file = '/content/valid_annotations.coco.json'
train_annotations = load_annotaion(train_annotation_file)
valid_annotations = load_annotaion(valid_annotation_file)
train_data = dataset_generator(train_image_dir, train_mask_dir,
                                train_annotations, batch_size,
                                target_size=(256, 256))
valid_data = dataset_generator(valid_image_dir, valid_mask_dir,
valid_annotations, batch_size, target_size=(256, 256))

In [ ]:
train_annotations['images']

In [ ]:
len(images)

In [ ]:
images , mask = next(train_data)
visiualize_batch(images,mask)
train_steps_per_epoch = len(train_annotations['images']) // batch_size
valid_steps_per_epoch = len(valid_annotations['images']) // batch_size

In [ ]:
print(train_steps_per_epoch)
print(valid_steps_per_epoch)

In [ ]:
model=unet_VGG16_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
history = model.fit(
 train_data,
 steps_per_epoch=train_steps_per_epoch,
 validation_data=valid_data,
 validation_steps=valid_steps_per_epoch,
 epochs=7,
 verbose=1
)
